In [27]:
import pandas as pd
import os
import numpy as np
from sklearn.cross_decomposition import CCA
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [20]:
#embs = pd.read_csv('/neurospin/dico/jlaval/Output/SC-sylv_12-16/16-38-32_170/troiani_custom_embeddings/custom_cross_val_embeddings.csv')
#embs = pd.read_csv('/neurospin/dico/jlaval/Output/SC-sylv_12-16/16-39-27_30/troiani_custom_embeddings/custom_cross_val_embeddings.csv')
#embs = pd.read_csv('/neurospin/dico/jlaval/Output/SC-sylv_left_v1/keep_bottom/troiani_custom_embeddings/custom_cross_val_embeddings.csv') ## V1 before even changing cutin prop
#embs = pd.read_csv('/neurospin/dico/jlaval/Output/SC-sylv_left_v1/no_keep_bottom/troiani_custom_embeddings/custom_cross_val_embeddings.csv') ## same here = best ?
#embs = pd.read_csv('/volatile/jl277509/Runs/02_STS_babies/Program/Output/SC-sylv_isomaps/16-40-54_148/troiani_embeddings/custom_cross_val_embeddings.csv')


In [23]:
## V0 with 256 dim
for k in range(5):
    embs = pd.read_csv(f'/volatile/jl277509/Runs/02_STS_babies/Program/Output/ISOMAP_CENTRAL/22-58-04_{k}/troiani_custom_embeddings/custom_cross_val_embeddings.csv')
    embs.columns=['Subject']+embs.columns[1:].tolist()
    embs_orig = embs.drop_duplicates(subset='Subject', keep='first')
    cols_embs = [f'dim{k}' for k in range(1,257)]

    # reformat embeddings and isomap
    isomap = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_isomap_labels.csv')
    merged = pd.merge(isomap, embs)
    cols_iso = [f'Isomap_central_left_dim{k}' for k in range(1,7)]
    embs, isomap = merged[cols_embs].to_numpy(), merged[cols_iso].to_numpy()

    # fit CCA and return score
    cca = CCA(n_components=len(cols_iso), scale=False) ## need to set scale to false !!
    cca.fit(embs, isomap)
    trans_embs, trans_isomap = cca.transform(embs, isomap)
    # Compute the canonical correlations between the canonical variables
    canonical_correlations = np.corrcoef(trans_embs.T, trans_isomap.T)[:trans_embs.shape[1], trans_isomap.shape[1]:]
    # Get the average of the canonical correlations
    average_correlation = np.mean(np.diagonal(canonical_correlations))

    # default sklearn score
    print(f'R2 score : {cca.score(embs, isomap)}')
    # custom score
    print(f'Average correlation : {average_correlation}')

    ## WHY WOULD CCA BE MUCH WORSE IF EACH R2 IS SIMILAR ??
    ## Not regularized, but I don't divide in train test...
    # should I split as train / test ?

R2 score : 0.1791501817258981
Average correlation : 0.9552178807047264
R2 score : 0.16115951622098967
Average correlation : 0.9575168623723367
R2 score : 0.1760009660158106
Average correlation : 0.9570704518519696


/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/cross_decomposition/_pls.py:113: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)


R2 score : 0.14660926525568807
Average correlation : 0.9559917160472278
R2 score : 0.16832458886068602
Average correlation : 0.9564313873421826


In [13]:
## V0 with 32 dim PCA
embs = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/SC-sylv_left/11-43-38_2/hcp_random_embeddings/full_embeddings.csv')
embs.columns=['Subject']+embs.columns[1:].tolist()
embs_orig = embs.drop_duplicates(subset='Subject', keep='first')
cols_orig = [f'dim{k}' for k in range(1,257)]
cols_embs = [f'dim{k}' for k in range(1,33)]

# perform PCA on the embeddings to keep 32 dims
pca = PCA(n_components=32)
embs = pca.fit_transform(embs_orig[cols_orig])
embs = pd.DataFrame(columns=cols_embs, data=embs)
embs['Subject']=embs_orig['Subject']

# reformat embeddings and isomap
isomap = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_isomap_labels.csv')
merged = pd.merge(isomap, embs)
cols_iso = [f'Isomap_central_left_dim{k}' for k in range(1,7)]
embs, isomap = merged[cols_embs].to_numpy(), merged[cols_iso].to_numpy()

# fit CCA and return score
cca = CCA(n_components=len(cols_iso), scale=False) ## need to set scale to false !!
cca.fit(embs, isomap)
projected_embs = cca.transform(embs)
print(cca.score(embs, isomap))

-51.72307740713265


In [26]:
# V1

model = 'all_augms_1'
epochs = [0,10,20,30]

for epoch in epochs:
    embs = pd.read_csv(f'/neurospin/dico/jlaval/Output/SC-sylv_left/{model}/hcp_isomap_random_epoch{epoch}_embeddings/full_embeddings.csv')

    # reformat embeddings and isomap
    isomap = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_isomap_labels.csv')
    embs.columns=['Subject']+embs.columns[1:].tolist()
    merged = pd.merge(isomap, embs)
    cols_embs = [f'dim{k}' for k in range(1,33)]
    cols_iso = [f'Isomap_central_left_dim{k}' for k in range(1,7)]
    embs, isomap = merged[cols_embs].to_numpy(), merged[cols_iso].to_numpy()

    # fit CCA and return score
    cca = CCA(n_components=len(cols_iso), scale=False) ## need to set scale to false !!
    cca.fit(embs, isomap)
    trans_embs, trans_isomap = cca.transform(embs, isomap)
    # Compute the canonical correlations between the canonical variables
    canonical_correlations = np.corrcoef(trans_embs.T, trans_isomap.T)[:trans_embs.shape[1], trans_isomap.shape[1]:]
    # Get the average of the canonical correlations
    average_correlation = np.mean(np.diagonal(canonical_correlations))

    # default sklearn score
    print(f'R2 score : {cca.score(embs, isomap)}')
    # custom score
    print(f'Average correlation : {average_correlation}')

R2 score : 0.1264129996679895
Average correlation : 0.6281532014674703
R2 score : 0.3732871000057694
Average correlation : 0.7442689941755797
R2 score : 0.42213828437062256
Average correlation : 0.7299465930197826
R2 score : 0.40206606449769183
Average correlation : 0.7285365769971334


# Need a train / test to account for overfitting

In [55]:
# V1
model='cutin_1'
embs = pd.read_csv(f'/neurospin/dico/jlaval/Output/SC-sylv_left/{model}/hcp_isomap_random_embeddings/full_embeddings.csv')
#embs = pd.read_csv('/neurospin/dico/jlaval/Output/SC-sylv_left_dim/preV1_32dims/hcp_isomap_random_embeddings/full_embeddings.csv')
n_dims = 32

# V0 # would require a regularisation to make it comparable to V1
# But what does it mean : is it better to trust CCA value of individual regressions ?
# with 256 dimensions, the CCA is overfitting, so it should be regularized
# If we need a large latent space to encompass the last isomap dimensions (because they could represent a small variance), then a regularization may not be enough,
# and a regression may be better suited.
#k=1
#embs = pd.read_csv(f'/volatile/jl277509/Runs/02_STS_babies/Program/Output/ISOMAP_CENTRAL/22-58-04_{k}/troiani_custom_embeddings/custom_cross_val_embeddings.csv')
#n_dims=256

# reformat embeddings and isomap
isomap = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_isomap_labels.csv')
embs.columns=['Subject']+embs.columns[1:].tolist()
merged = pd.merge(isomap, embs)
cols_embs = [f'dim{k}' for k in range(1,n_dims+1)]
cols_iso = [f'Isomap_central_left_dim{k}' for k in range(1,7)]
X, Y = merged[cols_embs].to_numpy(), merged[cols_iso].to_numpy()

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


# fit CCA and return score
cca = CCA(n_components=len(cols_iso), scale=False) ## need to set scale to false !!
cca.fit(X_train, Y_train)
# Transform the test data
X_c, Y_c = cca.transform(X_test, Y_test)
# Compute the canonical correlations between the canonical variables
canonical_correlations = np.corrcoef(X_c.T, Y_c.T)[:X_c.shape[1], X_c.shape[1]:]
# Get the average of the canonical correlations
average_correlation = np.mean(np.diagonal(canonical_correlations))

# default sklearn score
print(f'R2 score : {cca.score(X_test, Y_test)}')
# custom score
print(f'Average correlation : {average_correlation}')
print(f'Individual scores: {np.diagonal(canonical_correlations)}')

R2 score : 0.28638116749253384
Average correlation : 0.6527788790051248
Individual scores: [0.79685426 0.82249344 0.70803855 0.62203189 0.54318309 0.42407206]


In [ ]:
## keep it simple for the score as there are only 6 dims
## NB : results consistent with linear regression on the features
## NB : les dimensions des isomaps étant orthogonales, peut-être que la CCA n'apporte rien par rapport à des régressions linéaires sur chaque var.
# Mais c'est plus pratique d'avoir une seule valeur
# Devrait-on tenir compte de la taille d'effet de chaque isomap dans le score ? Ici on leur donne un poids identique.
# Mais la CKA est trop restrictive